## Detecção de objetos em vídeos com deep learning
***

A única diferença que vai existir é que ao inves de passar uma imagem, iremos passar um conjunto de imagens
que são os frames de um vídeo usando deep learning.

Instale o imutils: ```pip install imutils```

***
### Exemplo
***

In [1]:
import cv2 as cv
import numpy as np
import imutils
from imutils.video import VideoStream, FPS
from matplotlib import pyplot as plt

In [2]:
# Inicializar a lista de labels do mobilenet SSD
CLASSES = [
    "background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"
]

In [3]:
# Cria as cores para cada label ao inserir no imagem
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [4]:
# Carrega o modelo
modelo = cv.dnn.readNetFromCaffe(
    "./model/MobileNetSSD.prototxt",
    "./model/MobileNetSSD.caffemodel"
)

In [5]:
# Inicializa o vídeo stream, inicia o sensor da câmera e inicializa o FPS counter
# Para usar a webcam
video = VideoStream(src=0).start()
# Para usar uma câmera externa
# video = VideoStream(src=1).start()
# Para usar um vídeo
# video = VideoStream("video.mp4").start()

In [6]:
# Inicia a captura e contabiliza os FPS (frames por segundos)
fps = FPS().start()

In [7]:
# Loop sobre os frames do vídeo stream
while True:
    # Obtém o frame de vídeo stream e redimensiona para ter uma largura máxima de 900 pixels
    frame = video.read()
    frame = imutils.resize(frame, width=900)
    
    # Obtém as dimensões do frame e converte em um blob
    (altura, largura) = frame.shape[:2]
    blob = cv.dnn.blobFromImage(
        cv.resize(frame, (300, 300)),
        0.007843,
        (300, 300),
        127.5
    )
    
    # Passa o blob através da rede e obtém as detecções e previsões
    modelo.setInput(blob)
    deteccoes = modelo.forward()
    
    # Loop pelas detecções
    for i in np.arange(0, deteccoes.shape[2]):
        # Extrai a probabilidade associada com as previsões
        confianca = deteccoes[0, 0, i, 2]

        # Filtra detecções fracas garantindo que a confiança seja maior do que a confiança mínima
        confianca_minima = 0.9
        if confianca > confianca_minima:
            # Extrai o índice do rótulo de classe das detecções, então computa as coordenadas (x, y)
            # da caixa delimitadora para o objeto
            indice = int(deteccoes[0, 0, i, 1])
            caixa = deteccoes[0, 0, i, 3:7] * np.array([largura, altura, largura, altura])
            (startX, startY, endX, endY) = caixa.astype("int")

            # Desenha as previsões nos frames
            label = "{}: {:.2f}%".format(CLASSES[indice], confianca * 100)
            print("Label {}".format(label))
            cv.rectangle(frame, (startX, startY), (endX, endY), COLORS[indice], 2)

            if startY - 15 > 15:
                y = startY - 15
            else:
                y = startY + 15

            cv.putText(frame, label, (startX, y), cv.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[indice], 2)
        
    # Mostra a saída
    cv.imshow("Vídeo Stream", frame)
    key = cv.waitKey(1) & 0xFF
    
    # Se pressionada a tecla 'q' encerra o loop
    if key == ord('q'):
        break
        
    # Atualiza o FPS counter
    fps.update()

Label person: 98.81%
Label person: 96.52%
Label person: 96.52%
Label person: 90.63%
Label person: 92.26%
Label person: 93.28%
Label person: 93.62%
Label person: 90.78%
Label person: 91.06%
Label person: 94.51%
Label person: 92.44%
Label person: 92.99%
Label person: 95.11%
Label person: 95.11%
Label person: 92.86%
Label person: 92.33%
Label person: 92.33%
Label person: 93.12%
Label person: 93.12%
Label person: 94.57%
Label person: 97.57%
Label person: 97.57%
Label person: 93.78%


In [8]:
# Para o timer
fps.stop()
print("Tempo total de captura: {:.2f}".format(fps.elapsed()))
print("FPS: {:.2f}".format(fps.fps()))

Tempo total de captura: 6.98
FPS: 6.16


In [9]:
# Limpa tudo
cv.destroyAllWindows()
video.stop()